In [1]:
select * from INFORMATION_SCHEMA.COLUMNS

(279 rows affected)

Total execution time: 00:00:00.071

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,CHARACTER_OCTET_LENGTH,NUMERIC_PRECISION,NUMERIC_PRECISION_RADIX,NUMERIC_SCALE,DATETIME_PRECISION,CHARACTER_SET_CATALOG,CHARACTER_SET_SCHEMA,CHARACTER_SET_NAME,COLLATION_CATALOG,COLLATION_SCHEMA,COLLATION_NAME,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME
Denver Crime,dbo,Land_Transfer,PARID,1,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,RECEPTION_NUM,2,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,INSTRUMENT,3,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,SALE_YEAR,4,NULL,YES,float,NULL,NULL,53,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,SALE_MONTHDAY,5,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,RECEPTION_DATE,6,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,SALE_PRICE,7,NULL,YES,float,NULL,NULL,53,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,GRANTOR,8,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,GRANTEE,9,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL
Denver Crime,dbo,Land_Transfer,CLASS,10,NULL,YES,nvarchar,255,510,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,SQL_Latin1_General_CP1_CI_AS,NULL,NULL,NULL


In [11]:
SELECT top 15 *
FROM crime2010

(15 rows affected)

Total execution time: 00:00:00.006

incident_id,offense_id,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT
2010573988,2010573988131600,1316,0,threats-to-injure,public-disorder,2010-11-18 18:00:00.000,2010-11-18 18:10:00.000,2010-11-18 19:51:00.000,3100 BLK N LOWELL BLVD,3130861,1702583,-105.0345258,39.7614758,1,113,west-highland,1,0,1
2010474282,2010474282131600,1316,0,threats-to-injure,public-disorder,2010-09-21 11:30:00.000,NULL,2010-09-21 14:23:00.000,6801 E MISSISSIPPI AVE,3166178,1679337,-104.9094434,39.6970976,3,322,washington-virginia-vale,1,0,1
2010524977,2010524977131600,1316,0,threats-to-injure,public-disorder,2010-10-20 20:30:00.000,NULL,2010-10-20 21:28:00.000,2697 W ALAMEDA AVE,3135040,1684383,-105.0200033,39.7114536,4,411,valverde,1,0,1
2010434045,2010434045131600,1316,0,threats-to-injure,public-disorder,2010-08-29 08:00:00.000,2010-08-29 08:30:00.000,2010-08-29 10:11:00.000,2440 S MILWAUKEE ST,3154288,1670186,-104.9518928,39.6721806,3,314,university-park,1,0,2
2010436317,2010436317131600,1316,0,threats-to-injure,public-disorder,2010-08-30 10:00:00.000,2010-08-30 10:15:00.000,2010-08-30 16:51:00.000,9105 E LEHIGH AVE,3173893,1661749,-104.8824519,39.6486737,3,324,hampden-south,1,0,1
2010263502,2010263502131600,1316,0,threats-to-injure,public-disorder,2010-06-01 16:44:00.000,NULL,2010-06-01 17:43:00.000,2975 N HOLLY ST,3162328,1702148,-104.9226098,39.7597858,2,221,north-park-hill,1,0,1
2010528686,2010528686131600,1316,0,threats-to-injure,public-disorder,2010-10-22 21:00:00.000,NULL,2010-10-22 23:06:00.000,1430 N HUMBOLDT ST,3149267,1694479,-104.9692252,39.7389518,6,622,cheesman-park,1,0,1
2010573247,2010573247131600,1316,0,threats-to-injure,public-disorder,2010-11-18 10:15:00.000,2010-11-18 10:45:00.000,2010-11-18 11:16:00.000,1375 E 20TH AVE,3148743,1697661,-104.9710226,39.7476954,6,622,city-park-west,1,0,1
2010196209,2010196209131600,1316,0,threats-to-injure,public-disorder,2010-04-26 12:00:00.000,2010-04-26 12:05:00.000,2010-04-26 13:32:00.000,2285 S FEDERAL BLVD,3133498,1671467,-105.0257239,39.6760184,4,421,harvey-park,1,0,1
2010375092,2010375092131600,1316,0,threats-to-injure,public-disorder,2010-07-27 10:36:00.000,NULL,2010-07-28 08:30:00.000,2610 N FAIRFAX ST,3160844,1700407,-104.9279272,39.7550323,2,222,north-park-hill,1,0,1


In [3]:
-- Aggregate Crime dataset for the studied period (2010 - 2013)
WITH Crime_Data 
AS 
(
SELECT REPLACE(OFFENSE_CATEGORY_ID, '-', ' ') as OFFENSE_CATEGORY,
COUNT(*) as Count,
CONVERT(date, REPORTED_DATE) as REPORTED_DATE,
YEAR(REPORTED_DATE) as Year,
MONTH(REPORTED_DATE) as Month,
REPLACE(NEIGHBORHOOD_ID, '-', ' ') as NBHD_Name_Crime
FROM crime2010
GROUP BY REPLACE(OFFENSE_CATEGORY_ID, '-', ' '), CONVERT(date, REPORTED_DATE), YEAR(REPORTED_DATE), MONTH(REPORTED_DATE), REPLACE(NEIGHBORHOOD_ID, '-', ' ')

UNION ALL

SELECT REPLACE(OFFENSE_CATEGORY_ID, '-', ' ') as OFFENSE_CATEGORY,
COUNT(*) as Count,
CONVERT(date, REPORTED_DATE) as REPORTED_DATE,
YEAR(REPORTED_DATE) as Year,
MONTH(REPORTED_DATE) as Month,
REPLACE(NEIGHBORHOOD_ID, '-', ' ') as NBHD_Name_Crime
FROM crime2011
GROUP BY REPLACE(OFFENSE_CATEGORY_ID, '-', ' '), CONVERT(date, REPORTED_DATE), YEAR(REPORTED_DATE), MONTH(REPORTED_DATE), REPLACE(NEIGHBORHOOD_ID, '-', ' ')

UNION ALL

SELECT REPLACE(OFFENSE_CATEGORY_ID, '-', ' ') as OFFENSE_CATEGORY,
COUNT(*) as Count,
CONVERT(date, REPORTED_DATE) as REPORTED_DATE,
YEAR(REPORTED_DATE) as Year,
MONTH(REPORTED_DATE) as Month,
REPLACE(NEIGHBORHOOD_ID, '-', ' ') as NBHD_Name_Crime
FROM crime2012
GROUP BY REPLACE(OFFENSE_CATEGORY_ID, '-', ' '), CONVERT(date, REPORTED_DATE), YEAR(REPORTED_DATE), MONTH(REPORTED_DATE), REPLACE(NEIGHBORHOOD_ID, '-', ' ')
)

SELECT OFFENSE_CATEGORY, Year, Month, SUM(Count) as Count, NBHD_Name_Crime
INTO Crime_Data
FROM Crime_Data
WHERE REPORTED_DATE < '2013-01-01'
GROUP BY OFFENSE_CATEGORY, Year, Month, NBHD_Name_Crime
ORDER BY Year, Month

: Msg 2714, Level 16, State 6, Line 2
There is already an object named 'Crime_Data' in the database.

Total execution time: 00:00:00.034

In [10]:
SELECT top 15 *
FROM Crime_Data

(15 rows affected)

Total execution time: 00:00:00.002

OFFENSE_CATEGORY,Year,Month,Count,NBHD_Name_Crime
theft from motor vehicle,2010,9,2,regis
public disorder,2011,11,14,montbello
auto theft,2012,3,8,highland
auto theft,2011,4,3,mar lee
drug alcohol,2011,5,2,north capitol hill
white collar crime,2011,4,1,hampden south
burglary,2011,11,5,lincoln park
theft from motor vehicle,2011,10,2,skyland
white collar crime,2012,8,3,north capitol hill
drug alcohol,2010,5,2,gateway green valley ranch


In [9]:
SELECT top 15 *
FROM SalesBook

(15 rows affected)

Total execution time: 00:00:00.006

Model,Cluster,Group,CondoComplex,CondoComplexName,NBHD,PARCEL,ABSTRPRD,LIVEAREA,FINBSMNT,BASEMENT,YRBUILT,CONDITION,QUALITY,TOTUNITS,STORIES,GARTYPE,NOCARS,XTRAFFIC,FLOORLVL,NUMBDRM,NUMBATHS,COMMUSE,ARCSTYLE,SPRICE,DEDUCT,NETPRICE,TASP,SMONTH,SYEAR,QUALIFIED,STATUS
41,1,NULL,946,Wildhorse Ridge Condos,4,00102-08-065-065,1130,1346,0,0,2004,AVG,C,1,2,Att/Det,2,NULL,2,2,2,NULL,MIDDLE UNIT,100000,0,100000,100000,2,2012,Q,I
41,1,NULL,946,Wildhorse Ridge Condos,4,00102-08-073-073,1130,1249,0,0,2005,AVG,C,1,1,Att,1,NULL,1,2,2,NULL,END UNIT,94700,0,94700,94700,4,2011,Q,I
41,1,NULL,946,Wildhorse Ridge Condos,4,00102-08-078-078,1130,1346,0,0,2005,AVG,C,1,2,Att,1,NULL,2,2,2,NULL,MIDDLE UNIT,89500,0,89500,89500,10,2010,Q,I
41,1,NULL,946,Wildhorse Ridge Condos,4,00102-08-081-081,1130,1146,0,0,2005,AVG,C,1,1,None,0,NULL,2,2,2,NULL,END UNIT,92000,3220,88780,88780,10,2011,Q,I
41,1,NULL,946,Wildhorse Ridge Condos,4,00102-08-086-086,1130,1249,0,0,2005,AVG,C,1,1,Att,1,NULL,1,2,2,NULL,END UNIT,74199,0,74199,74199,3,2012,U,I
41,1,NULL,946,Wildhorse Ridge Condos,4,00102-08-086-086,1130,1249,0,0,2005,AVG,C,1,1,Att,1,NULL,1,2,2,NULL,END UNIT,105900,1000,104900,104900,5,2012,Q,I
41,1,NULL,946,Wildhorse Ridge Condos,4,00102-08-088-088,1130,1098,0,0,2005,AVG,C,1,2,None,0,NULL,2,1,1,NULL,MIDDLE UNIT,69000,0,69000,69000,8,2010,Q,I
41,1,NULL,946,Wildhorse Ridge Condos,4,00102-08-127-127,1130,1346,0,0,2005,AVG,C,1,2,Att,1,NULL,2,2,2,NULL,MIDDLE UNIT,90000,0,90000,90000,5,2012,Q,I
41,1,NULL,946,Wildhorse Ridge Condos,4,00102-08-133-133,1130,1249,0,0,2006,AVG,C,1,1,Att,1,NULL,1,2,2,NULL,END UNIT,100000,0,100000,100000,8,2010,Q,I
41,1,NULL,946,Wildhorse Ridge Condos,4,00102-08-141-141,1130,1346,0,0,2006,AVG,C,1,2,Att,1,NULL,2,2,2,NULL,MIDDLE UNIT,92000,0,92000,92000,12,2011,Q,I


In [8]:
SELECT top 15 *
FROM Land_Transfer

(15 rows affected)

Total execution time: 00:00:00.008

PARID,RECEPTION_NUM,INSTRUMENT,SALE_YEAR,SALE_MONTHDAY,RECEPTION_DATE,SALE_PRICE,GRANTOR,GRANTEE,CLASS,MKT_CLUS,D_CLASS,D_CLASS_N,NBHD_1,NBHD_1_CN
0232319031000,2017008883,SW,2017,0117,20170123,434927,SLOANS ROW LLC,"LAMEI,CHRISTOPHER H",H,58,194,RESIDENTIAL-ROWHOUSE,0247,E SLOANS LAKE
0232319031000,2020124751,WD,2020,0807,20200811,540000,"LAMEI,CHRISTOPHER H","RAMHOLD,CALVIN",H,58,194,RESIDENTIAL-ROWHOUSE,0247,E SLOANS LAKE
0232319032000,2015117254,PS,2015,0616,20150820,0,SLOANS LCP LLC,SLOANS LCP LLC,H,58,194,RESIDENTIAL-ROWHOUSE,0247,E SLOANS LAKE
0232319032000,2015149130,WD,2015,1020,20151022,0,SLOANS LCP LLC,SLOANS ROW LLC,H,58,194,RESIDENTIAL-ROWHOUSE,0247,E SLOANS LAKE
0232319032000,2017057561,WD,2017,0426,20170502,489900,SLOANS ROW LLC,"KOZIEL,THOMAS",H,58,194,RESIDENTIAL-ROWHOUSE,0247,E SLOANS LAKE
0232319032000,2021160717,SW,2021,0820,20210824,610000,"KOZIEL,THOMAS","LEE,ANGELA",H,58,194,RESIDENTIAL-ROWHOUSE,0247,E SLOANS LAKE
0232319033000,2015117254,PS,2015,0616,20150820,0,SLOANS LCP LLC,SLOANS LCP LLC,H,58,194,RESIDENTIAL-ROWHOUSE,0247,E SLOANS LAKE
0232319033000,2015149130,WD,2015,1020,20151022,0,SLOANS LCP LLC,SLOANS ROW LLC,H,58,194,RESIDENTIAL-ROWHOUSE,0247,E SLOANS LAKE
0232319033000,2017012542,WD,2017,0111,20170130,363889,SLOANS ROW LLC,"FARR,RACHEL",H,58,194,RESIDENTIAL-ROWHOUSE,0247,E SLOANS LAKE
0232319033000,2019090397,WD,2019,0711,20190712,400000,"FARR,RACHEL","OLSON,KERRY",H,58,194,RESIDENTIAL-ROWHOUSE,0247,E SLOANS LAKE


In [12]:
-- Aggregate crime neighborhood names across the studied period and join them to land trasfer dataset on neighborhood name
SELECT distinct L.NBHD_1 as NBHD_No, L.NBHD_1_CN AS NBHD_Name_Sales, C.NBHD_Name_Crime
FROM Crime_Data C
Full Join Land_Transfer L on L.NBHD_1_CN = C.NBHD_Name_Crime
order by NBHD_No

(324 rows affected)

Total execution time: 00:00:00.344

NBHD_No,NBHD_Name_Sales,NBHD_Name_Crime
NULL,NULL,NULL
NULL,NULL,barnum west
NULL,NULL,city park
NULL,NULL,city park west
NULL,NULL,college view south platte
NULL,NULL,dia
NULL,NULL,east colfax
NULL,NULL,elyria swansea
NULL,NULL,gateway green valley ranch
NULL,NULL,hale


the output is inspected in Excel to match the remaining neighborhood names to produce Neighborhood Mapping Key

In [13]:
SELECT *
FROM Neighborhood_Mapping

(294 rows affected)

Total execution time: 00:00:00.009

NBHD_No,NBHD_Name_Sales,NBHD_Name_Crime
1,MONTBELLO,montbello
1,MONTBELLO,dia
2,PARKFIELD,NULL
3,GREEN VALLEY,gateway green valley ranch
4,GATEWAY N,gateway green valley ranch
5,N GREEN VALLEY,gateway green valley ranch
6,C GREEN VALLEY,gateway green valley ranch
101,CENTRAL PARK,central park
102,CENTRAL PARK,central park
103,NE PARK HILL,northeast park hill


In [15]:
-- Join Real Estate Sales data on Crime data using the Neighborhood Mapping Key on Neighborhood ID & Name and on month and year
SELECT top 15 *
FROM (SELECT ROW_NUMBER() OVER (ORDER BY NBHD) AS SN, * FROM SalesBook) S
Left join Neighborhood_Mapping M on S.NBHD = CAST(M.NBHD_No AS float)
Inner join Crime_Data C on M.NBHD_Name_Crime = C.NBHD_Name_Crime
	and S.SMONTH = C.Month
	and S.SYEAR = C.Year
ORDER by SN

(15 rows affected)

Total execution time: 00:00:00.036

SN,Model,Cluster,Group,CondoComplex,CondoComplexName,NBHD,PARCEL,ABSTRPRD,LIVEAREA,FINBSMNT,BASEMENT,YRBUILT,CONDITION,QUALITY,TOTUNITS,STORIES,GARTYPE,NOCARS,XTRAFFIC,FLOORLVL,NUMBDRM,NUMBATHS,COMMUSE,ARCSTYLE,SPRICE,DEDUCT,NETPRICE,TASP,SMONTH,SYEAR,QUALIFIED,STATUS,NBHD_No,NBHD_Name_Sales,NBHD_Name_Crime,OFFENSE_CATEGORY,Year,Month,Count,NBHD_Name_Crime
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,auto theft,2011,9,5,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,sexual assault,2011,9,3,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,other crimes against persons,2011,9,9,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,aggravated assault,2011,9,2,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,all other crimes,2011,9,4,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,drug alcohol,2011,9,5,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,larceny,2011,9,17,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,theft from motor vehicle,2011,9,9,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,robbery,2011,9,6,montbello
1,41,1,NULL,634,Trails at Parkfield Lake-Phase I,1,00174-04-008-008,1130,925,0,0,2002,AVG,C,1,1,None,0,NULL,1,1,2,NULL,MIDDLE UNIT,55000,1500,53500,53500,9,2011,Q,I,1,MONTBELLO,montbello,burglary,2011,9,29,montbello
